# CBOW模型
CBOW：根据上下文之出现的关系去训练词向量的一种模型，CBOW根据上下文预测目标单词，最后使用模型的部分参数作为词向量。CBOW结构图如下：
<img src = "1.png" width = "500" height = "600"> <center>图1</center>
第一层为输入层：包含context(w)中上下文的２×win（窗口）个词向量。即对应目标单词w，选取其上下文各win个单词的词向量作为输入。
第二层为投影层：将输入层的２×win个向量做累加求和。
第三层为输出层：对应一颗二叉树，叶子节点共Ｎ个，对应词典里的每个词。 我们是通过哈弗曼树来求得某个词的条件概率的。假设某个词ｗ，从根节点出发到ｗ这个叶子节点，中间会经过４词分支，每一次分支都可以视为一次二分类。从二分类来说，word2ecv定义分到左边为负类（编码为１），分到右边为正类（编码label为０）。在逻辑回归中，一个节点被分为正类的概率为ｐ，分为负类的概率为１－ｐ。将每次分类的结果累乘则得到$p(w|Context(w))$
概率p在逻辑回归二分类问题中，对于任意样本$x=(x_1,x_2,x_3,...,x_n)^T $，利用sigmoid函数，求得分为正类的概率$h_{\theta}(w)=\sigma (\theta ^Tx)$负类概率为$1-h_{\theta}(w)=\sigma (\theta^Tx)$   
sigmoid函数为：$$\sigma (x) = \frac 1 {1+e^{-x}} \tag{公式1}$$

对于词典D中的任意词w，Huffman树中必存在一条从根节点到词w对应结点的路径$p^w$
 （且路径唯一），$p^w$路径上存在个$l^w-1$分支，每个分支看做一个二分类，每一次分类产生一个概率，将这些概率乘起来，就是所需的$p(w|Context(x))$
$$p(w|Context(x)) = \prod _{j=2}^{l^w} p(d_j ^{w} |X_w,\theta _{j-1}^w) \tag{公式2}$$
$d_j^w$为1或0,即某个单词在某条分支上的huffman编码  
$x_w$为输入词向量的求和平均
$\theta_{j-1}^w$为非叶子节点向量  
由上可知我们的目标函数就是$p(w|Context(w))$，但在基于神经网络的语言模型的目标函数通常取为如下对数似然函数：
$$\zeta = \sum_{w \in c}log \ p(w|Context(x)) \tag{公式3}$$

# 训练过程
tep1:准备好语料，将训练数据保存为txt文件中。
另取一些数据作为测试数据。
Step2:设置一个类class，保存词以及它的哈夫曼树路径、哈弗曼编码、词频。
Step3:初始化各类参数，扫描语料库，统计词频，并依据每个词的词频生成生成哈弗曼树。生成哈弗曼树后生成每个词的哈弗曼编码以及路径。初始化输入层词向量syn0以及哈弗曼树上非叶子结点的向量syn1。
Step4:训练，迭代优化。 训练过程中就是通过不断的输入，用随机梯度上升的方法，去更新词向量的值（syn0），非叶子结点处向量的值(syn1)。实质上就是让词向量在词向量空间中找到正确的位置。
训练伪代码如图：<img src = "2.png">
Step5：训练完成后进行预测
将目标词上下文本作为输入，获得词向量后相加平均得$x_w $求关键字路径哈夫曼编码，
将$x_w$与$syn1（\theta_{j-1}^w)$做乘积的结果再经sigmoid函数得概率q。根据二分类，求得概率P。累乘即可得目标词概率，即公式2：
$$p(w|Context(x)) = \prod _{j=2}^{l^w} p(d_j ^{w} |X_w,\theta _{j-1}^w) \tag{公式2}$$